In [28]:
import tensorflow as tf          
import pathlib
import re
import numpy as np 

from keras import layers as L                   
from keras.applications import MobileNetV2      
from keras.applications.mobilenet_v2 import (
    preprocess_input,                           
)


In [29]:
DATA_ROOT   = pathlib.Path("../Datasets")          # <- adjust if needed
IMG_SIZE    = (224, 224)                        # images are 224×224×3
BATCH_SIZE  = 32
EPOCHS      = 15
SEED        = 42

# --- 1. Collect file paths ----------------------------------------------------
train_files, train_labels = [], []
test_files , test_labels  = [], []

class_names = sorted([p.name for p in DATA_ROOT.iterdir() if p.is_dir()])
class_to_idx = {c: i for i, c in enumerate(class_names)}

pattern = re.compile(r"(\d+)\.(jpg|jpeg|png)$", re.IGNORECASE)

for cls in class_names:
    for fp in (DATA_ROOT / cls).iterdir():
        m = pattern.search(fp.name)
        if not m:                    # skip non-image files
            continue
        idx = int(m.group(1))
        target = test_files if idx % 5 == 0 else train_files
        labels = test_labels if idx % 5 == 0 else train_labels
        target.append(str(fp))
        labels.append(class_to_idx[cls])

print(f"Train images: {len(train_files)}, Test images: {len(test_files)}")

Train images: 692, Test images: 170


In [30]:
# --- 2. Build tf.data datasets ------------------------------------------------
def make_ds(paths, labels, training=False):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    def _load(f, y):
        img = tf.io.read_file(f)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, IMG_SIZE)
        img = preprocess_input(img)            # MobileNetV2 preprocessing
        return img, tf.one_hot(y, len(class_names))

    ds = ds.map(_load, num_parallel_calls=tf.data.AUTOTUNE)
    if training:
        aug = tf.keras.Sequential([
            L.RandomFlip("horizontal"),
            L.RandomRotation(0.05),
            L.RandomZoom(0.1)
        ])
        ds = ds.map(lambda x, y: (aug(x, training=True), y),
                    num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.shuffle(1024, seed=SEED)
    return ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_ds = make_ds(train_files, train_labels, training=True)
test_ds  = make_ds(test_files , test_labels , training=False)

In [31]:
# --- 3. Build & compile the model --------------------------------------------
base = MobileNetV2(input_shape=IMG_SIZE + (3,), weights="imagenet",
                   include_top=False)
base.trainable = False                       # first train only the head

model = tf.keras.Sequential([
    base,
    L.GlobalAveragePooling2D(),
    L.Dropout(0.25),
    L.Dense(len(class_names), activation="softmax")
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="categorical_crossentropy",
              metrics=["accuracy"])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [32]:
# --- 4. Train (head only) -----------------------------------------------------
hist = model.fit(train_ds,
                 epochs=EPOCHS,
                 validation_data=test_ds,
                 callbacks=[
                     tf.keras.callbacks.EarlyStopping(patience=3,
                                                     restore_best_weights=True)
                 ])

Epoch 1/15


I0000 00:00:1750432736.901079   13489 service.cc:146] XLA service 0x7af3b00029f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750432736.901099   13489 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6
2025-06-20 23:18:56.956298: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-06-20 23:18:57.259907: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90501


11/22 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3124 - loss: 1.8367

I0000 00:00:1750432741.172332   13489 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 0.3783 - loss: 1.6724

2025-06-20 23:19:05.797363: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1179', 100 bytes spill stores, 100 bytes spill loads

2025-06-20 23:19:05.852269: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1179', 340 bytes spill stores, 340 bytes spill loads

2025-06-20 23:19:07.229623: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1179', 60 bytes spill stores, 60 bytes spill loads



22/22 ━━━━━━━━━━━━━━━━━━━━ 15s 405ms/step - accuracy: 0.3825 - loss: 1.6613 - val_accuracy: 0.7059 - val_loss: 0.8630
Epoch 2/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.6565 - loss: 0.8835 - val_accuracy: 0.7824 - val_loss: 0.6045
Epoch 3/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8037 - loss: 0.5369 - val_accuracy: 0.8353 - val_loss: 0.5081
Epoch 4/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8173 - loss: 0.5388 - val_accuracy: 0.8471 - val_loss: 0.4653
Epoch 5/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8488 - loss: 0.4506 - val_accuracy: 0.8529 - val_loss: 0.4116
Epoch 6/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8928 - loss: 0.3661 - val_accuracy: 0.8941 - val_loss: 0.3978
Epoch 7/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8782 - loss: 0.3751 - val_accuracy: 0.9059 - val_loss: 0.3607
Epoch 8/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9101 - loss: 0.2935 - val_accuracy: 0.9059 - val_loss: 

In [33]:
# --- 5. Optional fine-tuning ---------------------------------------------------
base.trainable = True
for layer in base.layers[:-20]:              # unfreeze last ~20 layers only
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_ds,
          epochs=EPOCHS,
          validation_data=test_ds,
          callbacks=[
              tf.keras.callbacks.EarlyStopping(patience=3,
                                               restore_best_weights=True)
          ])

Epoch 1/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 248ms/step - accuracy: 0.8684 - loss: 0.4016 - val_accuracy: 0.8882 - val_loss: 0.3625
Epoch 2/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9561 - loss: 0.1580 - val_accuracy: 0.9000 - val_loss: 0.3140
Epoch 3/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9826 - loss: 0.0825 - val_accuracy: 0.8882 - val_loss: 0.3180
Epoch 4/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9849 - loss: 0.0701 - val_accuracy: 0.9059 - val_loss: 0.3051
Epoch 5/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9936 - loss: 0.0460 - val_accuracy: 0.8941 - val_loss: 0.3239
Epoch 6/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9878 - loss: 0.0483 - val_accuracy: 0.9000 - val_loss: 0.2610
Epoch 7/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9931 - loss: 0.0377 - val_accuracy: 0.9353 - val_loss: 0.2135
Epoch 8/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 1.0000 - loss: 0.0255 - val_accuracy: 0.9353 -

In [34]:
# --- 6. Inspect mis-classifications ------------------------------------------
y_true, y_pred, file_paths = [], [], []

for (x, y), paths in zip(test_ds.unbatch(), np.array(test_files)):
    logits = model(tf.expand_dims(x, 0), training=False)
    y_true.append(tf.argmax(y).numpy())
    y_pred.append(tf.argmax(logits, axis=1).numpy()[0])
    file_paths.append(paths)

mis_idx = [i for i, (t, p) in enumerate(zip(y_true, y_pred)) if t != p]
print("\nMis-classified samples (truth -> pred):")
for i in mis_idx:
    print(f"{file_paths[i]} : {class_names[y_true[i]]} → {class_names[y_pred[i]]}")


W0000 00:00:1750432792.091046   12795 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750432792.104512   12795 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750432792.105091   12795 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750432792.105655   12795 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750432792.106195   12795 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750432792.106742   12795 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750432792.107296   12795 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750432792.107836   12795 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1750432792.109722   12795 gp


Mis-classified samples (truth -> pred):
../Datasets/Bacterial_Leaf_Blight/15.jpeg : Bacterial_Leaf_Blight → Neck_Blast
../Datasets/False_Smut/60.jpeg : False_Smut → Neck_Blast
../Datasets/False_Smut/90.jpeg : False_Smut → Healthy
../Datasets/False_Smut/80.jpeg : False_Smut → Neck_Blast
../Datasets/Healthy/45.jpeg : Healthy → Brown_Spot


2025-06-20 23:19:59.899954: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
